# Cirq Basics

This notebook loosely follows the introduction to the documentation (https://cirq.readthedocs.io/en/stable/) with a few deviations here and there.

In [1]:
import cirq

In [2]:
print(cirq.__version__)

0.6.0


Cirq builds a circuit through building a grid made of moments. Moments can be thought of as steps in time through the circuit. Each moment is a combination of operations, which include gates or measurements. 

In order to construct our circuit, we must first define the grid length. The operation of GridQubit is the construction of a qubit on a 2d square lattice. Alternatively, we could instantiate a LineQubit system, which just puts the qubits in a line rather than a lattice.

In [3]:
length = 3

# We now loop through the length to create a 3x3 qubit grid.
qubits = [cirq.GridQubit(i, j) for i in range(length) for j in range(length)]
print(qubits)

[cirq.GridQubit(0, 0), cirq.GridQubit(0, 1), cirq.GridQubit(0, 2), cirq.GridQubit(1, 0), cirq.GridQubit(1, 1), cirq.GridQubit(1, 2), cirq.GridQubit(2, 0), cirq.GridQubit(2, 1), cirq.GridQubit(2, 2)]


We can reference a qubit as you would expect in a python array.

In [4]:
qubits[1]

cirq.GridQubit(0, 1)

We can do many things with this grid now. One important thing for Cirq is checking if two qubits are adjacent. We can do this with the below code. 

In [5]:
# Check the first and second qubits in our qubits list:
#  These correspond to the (0,0) and the (0,1) placements
print(qubits[1].is_adjacent(qubits[2]))

# Check the first and third qubits in our qubits list:
#  These correspond to the (0,0) and the (0,2) placements
print(qubits[1].is_adjacent(qubits[3]))

True
False


So, we can see that our grid truly respects the intuitive connectivity of the lattice. This has implications on our code because during a two qubit gate we can only manipulate two qubits that are adjacent. Thus, we may have to incorporate swap gates in our final circuit in order to perform the circuit correctly.

In [6]:
for i in range(length*length): 
  for j in range(length*length):
    if j==i:
      continue
    print("Qubits {} and {}: ".format(i,j)+str(qubits[i].is_adjacent(qubits[j])))

Qubits 0 and 1: True
Qubits 0 and 2: False
Qubits 0 and 3: True
Qubits 0 and 4: False
Qubits 0 and 5: False
Qubits 0 and 6: False
Qubits 0 and 7: False
Qubits 0 and 8: False
Qubits 1 and 0: True
Qubits 1 and 2: True
Qubits 1 and 3: False
Qubits 1 and 4: True
Qubits 1 and 5: False
Qubits 1 and 6: False
Qubits 1 and 7: False
Qubits 1 and 8: False
Qubits 2 and 0: False
Qubits 2 and 1: True
Qubits 2 and 3: False
Qubits 2 and 4: False
Qubits 2 and 5: True
Qubits 2 and 6: False
Qubits 2 and 7: False
Qubits 2 and 8: False
Qubits 3 and 0: True
Qubits 3 and 1: False
Qubits 3 and 2: False
Qubits 3 and 4: True
Qubits 3 and 5: False
Qubits 3 and 6: True
Qubits 3 and 7: False
Qubits 3 and 8: False
Qubits 4 and 0: False
Qubits 4 and 1: True
Qubits 4 and 2: False
Qubits 4 and 3: True
Qubits 4 and 5: True
Qubits 4 and 6: False
Qubits 4 and 7: True
Qubits 4 and 8: False
Qubits 5 and 0: False
Qubits 5 and 1: False
Qubits 5 and 2: True
Qubits 5 and 3: False
Qubits 5 and 4: True
Qubits 5 and 6: False
Qubi

Now that we have our qubits created, we should probably do things with them. We can now have gates act *on* the qubit. The documentation emphasises that the gate acts *on* a qubit. What are some of the gates we can use? Well the basic ones are:

- cirq.H The Hadamard gate.
- cirq.I The one qubit identity gate.
- cirq.S The Clifford S gate.
- cirq.T The non-Clifford T gate.
- cirq.X The Pauli X gate.
- cirq.Y The Pauli Y gate.
- cirq.Z The Pauli Z gate.
- cirq.CX The controlled NOT gate.
- cirq.CZ The controlled Z gate.
- cirq.XX The tensor product of two X gates.
- cirq.YY The tensor product of two Y gates.
- cirq.ZZ The tensor product of two Z gates.
- cirq.rx(rads) Returns a gate with the matrix $e^{-i X rads / 2}$.
- cirq.ry(rads) Returns a gate with the matrix $e^{-i Y rads / 2}$.
- cirq.rz(rads) Returns a gate with the matrix $e^{-i Z rads / 2}$.
- cirq.CCNOT The TOFFOLI gate.
- cirq.CCX The TOFFOLI gate.

With many more available in the documentation.

This is further clear in the code below.

In [7]:
# We define the X gate with the next line
x_gate = cirq.X

# Then we specify that this will act on our 0-index qubit by defining
# the operation as a variable
x_op = x_gate(qubits[0])
print(x_op)

X((0, 0))


Now, we can specify gates acting on multiple qubits and combine them to create a moment, which can be a specific order of a circuit, but does not need to be.

In [8]:
# For a controlled gate the first qubit specified is the control and the second
#   is the target
cz = cirq.CZ(qubits[0], qubits[1])
x = cirq.X(qubits[2])
moment = cirq.Moment([x, cz])

print(moment)

X((0, 2)) and CZ((0, 0), (0, 1))


If we try to specify two different operations on the same qubit we get an error. Thus, when we specify a moment, it must be on a disjoint set of qubits.

If a moment is the combination of multiple operations, a circuit is a combination of multiple moments. Additionally, a circuit is an ordered series of moments with the first moment occuring first. 

We will use *OP_TREE*s to build circuits/moments. An OP_TREE is not a class, but a contract. The basic idea is that, if the input can be iteratively flattened into a list of operations, then the input is an OP_TREE. Examples are:

- A single Operation.
- A list of Operations.
- A tuple of Operations.
- A list of a list of Operationss.
- A generator yielding Operations.

We now entangle two qubits.

In [9]:
cx12 = cirq.CX(qubits[1], qubits[2])
H1 = cirq.H(qubits[1])
moment0 = cirq.Moment([H1])
moment1 = cirq.Moment([cx12])
circuit = cirq.Circuit((moment0, moment1))
print(circuit)

(0, 1): ───H───@───
               │
(0, 2): ───────X───


Some other ways to create circuits are shown below

In [10]:
# Import the gates to be used
from cirq.ops import CX, CZ, H, X, I

# Name each qubit that is desired.
q0, q1 = [cirq.GridQubit(i, 0) for i in range(2)]

# Initiate the circuit and name it
circuit = cirq.Circuit()

# Append the operations to the circuit and the circuit
# will be constructed with the ordering 
# InsertStrategy.NEW_THEN_INLINE by default.
# We insert the identity gate
# to demonstrate this.
circuit.append([H(q0),I(q1),CX(q0, q1)])
print(circuit)

(0, 0): ───H───@───
               │
(1, 0): ───I───X───


We have now specified the same circuit in fewer lines and less tedious coding.

If we do not want to have the new then inline insert strategy, we can specify a different one. There are four strategies: 
1. InsertStrategy.EARLIEST: 
2. InsertStrategy.NEW, 
3. InsertStrategy.INLINE and 
4. InsertStrategy.NEW_THEN_INLINE. (default)

EARLIEST scans backward from the insert location until a moment with operations touching qubits affected by the operation to insert is found. The operation is added into the moment just after that location. We demonstrate this with the below

In [11]:
from cirq.circuits import InsertStrategy
circuit = cirq.Circuit()
q0, q1, q2 = [cirq.GridQubit(i, 0) for i in range(3)]
circuit.append([CZ(q0, q1)])
circuit.append([H(q0), H(q2)], strategy=InsertStrategy.EARLIEST)
print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───────

(2, 0): ───H───────


As we can see because there was already an operation acting on qubit 0, so our new hadamard gate on q0 was appended after it; however, there was no operation on qubit 2 so it was placed in the first moment.

We now use the same code with the NEW strategy

In [12]:
circuit = cirq.Circuit()
circuit.append([CZ(q0, q1)])
circuit.append([H(q0), H(q2)], strategy=InsertStrategy.NEW)
print(circuit)

(0, 0): ───@───H───────
           │
(1, 0): ───@───────────

(2, 0): ───────────H───


We can see that this appends each operation into a new moment. Now, we see what happens if we use the INLINE strategy.

In [13]:
circuit = cirq.Circuit()
circuit.append([CZ(q0, q1)])
circuit.append([H(q0), H(q2)], strategy=InsertStrategy.INLINE)
print(circuit)

(0, 0): ───@───H───
           │
(1, 0): ───@───────

(2, 0): ───────H───


The circuit then attempts to add the operation to insert into the moment just before the
desired insert location. But, if there’s already an existing operation affecting any of the qubits touched by
the operation to insert, a new moment is created instead. We will look at another example for this one.

In [14]:
circuit = cirq.Circuit()
circuit.append([CZ(q1, q2)])
circuit.append([CZ(q1, q2)])
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.INLINE)
print(circuit)

(0, 0): ───────H───────

(1, 0): ───@───@───H───
           │   │
(2, 0): ───@───@───H───


*Think through what happens if you change the insert strategy for the above circuit*

Now, what exactly does our default strategy do? It Creates a new moment at the desired insert location for the first operation, but then switches to inserting operations according to InsertStrategy.INLINE. We see this in the below.

In [15]:
circuit = cirq.Circuit()
circuit.append([H(q0)])
circuit.append([CZ(q1,q2), H(q0)], strategy=InsertStrategy.NEW_THEN_INLINE)
print(circuit)

(0, 0): ───H───H───

(1, 0): ───────@───
               │
(2, 0): ───────@───


We can do more than just specify lists to our append operation. We can also write a function to pass through the append. 

In [16]:
def my_layer():
    yield CZ(q0, q1)
    yield [H(q) for q in (q0, q1, q2)]
    yield [CZ(q1, q2)]
    yield [H(q0), [CZ(q1, q2)]]
circuit = cirq.Circuit()
circuit.append(my_layer())
print("The moments are:")
for x in my_layer():
    print(x)
print("\nThe circuit is:")
print(circuit)

The moments are:
CZ((0, 0), (1, 0))
[cirq.H.on(cirq.GridQubit(0, 0)), cirq.H.on(cirq.GridQubit(1, 0)), cirq.H.on(cirq.GridQubit(2, 0))]
[cirq.CZ.on(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]
[cirq.H.on(cirq.GridQubit(0, 0)), [cirq.CZ.on(cirq.GridQubit(1, 0), cirq.GridQubit(2, 0))]]

The circuit is:
(0, 0): ───@───H───H───────
           │
(1, 0): ───@───H───@───@───
                   │   │
(2, 0): ───H───────@───@───


We can also put our OP_TREE right into the circuit when we initialize it.

In [17]:
circuit = cirq.Circuit(my_layer())
print(circuit)

(0, 0): ───@───H───H───────
           │
(1, 0): ───@───H───@───@───
                   │   │
(2, 0): ───H───────@───@───


We can reference qubits by their row index (using <code>.row</code>) or column index (using <code>.col</code>).

In [18]:
circuit = cirq.Circuit()
circuit.append([cirq.H(q) for q in qubits if (q.row + q.col) % 2 == 0],
               strategy=cirq.InsertStrategy.EARLIEST)
circuit.append([cirq.X(q) for q in qubits if (q.row + q.col) % 2 == 1],
               strategy=cirq.InsertStrategy.NEW_THEN_INLINE)
print(circuit)

(0, 0): ───H───────

(0, 1): ───────X───

(0, 2): ───H───────

(1, 0): ───────X───

(1, 1): ───H───────

(1, 2): ───────X───

(2, 0): ───H───────

(2, 1): ───────X───

(2, 2): ───H───────


### Iterating and slicing

We can also iterate and slice circuits. When we iterate over a circuit, each iterate is a moment.

In [19]:
circuit = cirq.Circuit(H(q0), CZ(q0, q1))
for moment in circuit:
    print(moment)

H((0, 0))
CZ((0, 0), (1, 0))


We can then slice a circuit into moments.

In [20]:
circuit = cirq.Circuit(H(q0), CZ(q0, q1), H(q1), CZ(q0, q1))
print(circuit)
print("\nTaking the slice at index 1 and 2:")
print(circuit[1:3]) # Remember to use python indexing for the array.

(0, 0): ───H───@───────@───
               │       │
(1, 0): ───────@───H───@───

Taking the slice at index 1 and 2:
(0, 0): ───@───────
           │
(1, 0): ───@───H───


We now see what this looks like based on the different insert strategies.

In [21]:
circuit = cirq.Circuit()
circuit.append([CZ(q1, q2)])
circuit.append([CZ(q1, q2)])
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.INLINE)
print(circuit)
print("\nTaking the slice at index 1:")
print(circuit[1:2])

(0, 0): ───────H───────

(1, 0): ───@───@───H───
           │   │
(2, 0): ───@───@───H───

Taking the slice at index 1:
(0, 0): ───H───

(1, 0): ───@───
           │
(2, 0): ───@───


In [22]:
circuit = cirq.Circuit()
circuit.append([CZ(q1, q2)])
circuit.append([CZ(q1, q2)])
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.NEW)
print(circuit)
print("\nTaking the slice at index 1:")
print(circuit[1:2])

(0, 0): ───────────H───────────

(1, 0): ───@───@───────H───────
           │   │
(2, 0): ───@───@───────────H───

Taking the slice at index 1:
(1, 0): ───@───
           │
(2, 0): ───@───


In [23]:
circuit = cirq.Circuit()
circuit.append([CZ(q1, q2)])
circuit.append([CZ(q1, q2)])
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.EARLIEST)
print(circuit)
print("\nTaking the slice at index 1:")
print(circuit[1:2])

(0, 0): ───H───────────

(1, 0): ───@───@───H───
           │   │
(2, 0): ───@───@───H───

Taking the slice at index 1:
(1, 0): ───@───
           │
(2, 0): ───@───


Especially useful is dropping the last moment (which are often just measurements according to the documentation): <code>circuit[:-1]</code>, or reversing a
circuit: <code>circuit[::-1]</code>.

In [24]:
circuit = cirq.Circuit()
circuit.append([CZ(q1, q2)])
circuit.append([CZ(q1, q2)])
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.NEW)
print(circuit)
print("\nReversing the circuit:")
print(circuit[::-1])

(0, 0): ───────────H───────────

(1, 0): ───@───@───────H───────
           │   │
(2, 0): ───@───@───────────H───

Reversing the circuit:
(0, 0): ───────────H───────────

(1, 0): ───────H───────@───@───
                       │   │
(2, 0): ───H───────────@───@───


We can also decompose moements into smaller 1 and 2 qubit gates. Though there are quite technical caveats to the current abilities of the cirq decomposition.

In [25]:
from cirq.ops import CCX
circuit = cirq.Circuit()
circuit.append([CCX(q0, q1, q2)])
print(circuit)
print("\nDecomposition:")
circuit2 = cirq.Circuit()
for moment in circuit:
    circuit2.append(cirq.decompose(moment))
print(circuit2)

(0, 0): ───@───
           │
(1, 0): ───@───
           │
(2, 0): ───X───

Decomposition:
(0, 0): ───T────────────────@─────────────────────────────────@─────────────────────────────@────────────────────────────@───────────────────────────────────────
                            │                                 │                             │                            │
(1, 0): ───T───────Y^-0.5───@───Y^0.5────@───T^-1────Y^-0.5───@────────Y^0.5───@───Y^-0.5───@──────Y^0.5────@───Y^-0.5───@──────Y^0.5────@───────────────────────
                                         │                                     │                            │                            │
(2, 0): ───Y^0.5───X────────T───Y^-0.5───@───Y^0.5───T────────Y^-0.5───────────@───Y^0.5────T^-1───Y^-0.5───@───Y^0.5────T^-1───Y^-0.5───@───Y^0.5───Y^0.5───X───


In [26]:
circuit = cirq.Circuit()
circuit.append([CZ(q1, q2)])
circuit.append([CZ(q1, q2)])
circuit.append([H(q0), H(q1), H(q2)], strategy=InsertStrategy.NEW)
circuit2 = cirq.Circuit()
print(circuit)
print("\nDecomposition:")
for moment in circuit:
    circuit2.append(cirq.decompose(moment))
print(circuit2)

(0, 0): ───────────H───────────

(1, 0): ───@───@───────H───────
           │   │
(2, 0): ───@───@───────────H───

Decomposition:
(0, 0): ───Y^0.5───X───────────────

(1, 0): ───@───────@───Y^0.5───X───
           │       │
(2, 0): ───@───────@───Y^0.5───X───


## Simulation

The simulators make a distrinction between a "run" and a "simulation". A "run" only allows for a simulation that mimics the actual quantum hardware. For example, it does not allow for access to the amplitudes of the wave function of the system. "Simulate" commands are more broad and allow different forms of simulation. One should be wary of relying on "simulate" methods when run against actual hardware.

To run a simulation of the full circuit we simply create a simulator, and pass the circuit to the simulator.

In [27]:
from cirq.ops import measure

# Initiate the simulator and the circuit
sim = cirq.Simulator()
circuit = cirq.Circuit()

# Define the qubits
q1, q2, q3 = [cirq.LineQubit(i) for i in range(3)]

# Define the moments
def my_layers():
    yield [X(q1)]
    yield [H(q2)]
    yield [CX(q2,q3)]
    yield measure(q1, key='q1'), measure(q2, key='q2'), measure(q3, key='q3')
    
# Add the gates to the circuit
circuit.append(my_layers())

# Run the simulation using 'simulate'
results = sim.simulate(circuit)
print("Simulation: {}".format(i))
print(results)

Simulation: 8
measurements: q1=1 q2=0 q3=0
output vector: |100⟩


This is neat! We see that we get the measurement readout from each qubit as well as the output vector in bra-ket notation.

What else can we get? We can also get the Bloch Sphere coordinates.

In [28]:
# Print the Bloch Sphere coordinates. Cirq has a function that
#    returns the coordinates as a vector. The first argument is 
#    the state you want to check, and the second argument is the 
#    index of the qubit you want to show.

bX, bY, bZ = cirq.bloch_vector_from_state_vector(results.final_state, 0)
print("x: ", round(bX, 4),
      "\ny: ", round(bY, 4),
      "\nz: ", round(bZ, 4))

x:  0.0 
y:  0.0 
z:  -1.0


In [29]:
# Print the Bloch Sphere coordinates. Cirq has a function that
#    returns the coordinates as a vector. The first argument is 
#    the state you want to check, and the second argument is the 
#    index of the qubit you want to show.

bX, bY, bZ = cirq.bloch_vector_from_state_vector(results.final_state, 1)
print("x: ", round(bX, 4),
      "\ny: ", round(bY, 4),
      "\nz: ", round(bZ, 4))

x:  0.0 
y:  0.0 
z:  1.0


In [30]:
# Print the Bloch Sphere coordinates. Cirq has a function that
#    returns the coordinates as a vector. The first argument is 
#    the state you want to check, and the second argument is the 
#    index of the qubit you want to show.

bX, bY, bZ = cirq.bloch_vector_from_state_vector(results.final_state, 2)
print("x: ", round(bX, 4),
      "\ny: ", round(bY, 4),
      "\nz: ", round(bZ, 4))

x:  0.0 
y:  0.0 
z:  1.0


Now, let's compare what happens if we use 'run' instead. This time, we can call the number of repetitions in the arguments of the function.

In [31]:
results = sim.run(circuit, repetitions=100)
print(results)

q1=1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111
q2=0111001101010111101101101010100100101010001101000111100000000010101000001010110011010011011001001001
q3=0111001101010111101101101010100100101010001101000111100000000010101000001010110011010011011001001001


We can see that the second and third qubits are always the same! Which is exactly what we expect.

We can also get some summary statistics with the below code.

In [32]:
print(results.histogram(key='q1'))
print(results.histogram(key='q2'))
print(results.histogram(key='q3'))

Counter({1: 100})
Counter({0: 54, 1: 46})
Counter({0: 54, 1: 46})


We can also see that roughly half the time the second and third qubits are either one or zero, which is exactly what we expect given they are in the bell state.